In [8]:
import os

root = "../data/processed/bzd"

for folder in sorted(os.listdir(root)):
    path = os.path.join(root, folder)
    print(f"{path}")

../data/processed/bzd/BerolPhill1516
../data/processed/bzd/BerolPhill1639
../data/processed/bzd/albini1520
../data/processed/bzd/albini1549
../data/processed/bzd/katelos1420
../data/processed/bzd/katelos1506
../data/processed/bzd/katelos1626


In [1]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1)),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.ColorJitter(brightness=0.2, contrast=0.3),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.ToTensor()
])